In [2]:
client_id = 
client_secret = 

import pandas as pd
from datetime import datetime, timedelta
import requests
import re
def handle_authorization(data):
    
    access_token = data["access_token"]
    print(f" {access_token}")
    return access_token
    
def get_twitch_authorization(client_id, client_secret):
    url = f"https://id.twitch.tv/oauth2/token"
    params = {
        "client_id": client_id,
        "client_secret": client_secret,
        "grant_type": "client_credentials",
    }
    
    response = requests.post(url, params=params)
    data = response.json()
    
    handle_authorization(data)



# Obtén la autorización de Twitch
get_twitch_authorization(client_id, client_secret)

 qzshhrkynf4awcc7owly9g5u4wfz2x


In [3]:
oauth_token = 

In [4]:
def extract_channel_name(url):
    pattern = r"https?:\/\/(?:www\.|go\.)?twitch\.tv\/([a-zA-Z0-9_]{4,25})"
    match = re.search(pattern, url)
    if match:
        return match.group(1)
    return None
def get_streamer_data(client_id, url):
    channel_name = extract_channel_name(url)
    if not channel_name:
        return None
    
    url = f"https://api.twitch.tv/helix/users"
    
    params = {
        "login": channel_name
    }
    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }
    
    response = requests.get(url, params=params, headers=headers)
    data = response.json()
    print(data)
    if "data" in data and len(data["data"]) > 0:
        user_id = data["data"][0]["id"]
        stream_url = f"https://api.twitch.tv/helix/streams?user_id={user_id}"
        response = requests.get(stream_url, headers=headers)
        stream_data = response.json()
        
        if "data" in stream_data and len(stream_data["data"]) > 0:
            return stream_data["data"][0]
    
    return None
url = "https://www.twitch.tv/ibai"
streamer_data = get_streamer_data(client_id, url)

if streamer_data:
    data = f"Datos del streamer {channel_url}:"

{'data': [{'id': '83232866', 'login': 'ibai', 'display_name': 'ibai', 'type': '', 'broadcaster_type': 'partner', 'description': 'Si lees esto que sepas que te aprecio', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/574228be-01ef-4eab-bc0e-a4f6b68bedba-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/b01927d9-1cc2-4ba0-b3e2-6e96959179d0-channel_offline_image-1920x1080.jpeg', 'view_count': 0, 'created_at': '2015-02-20T16:47:56Z'}]}


In [5]:
data = {'data': [{'id': '83232866', 'login': 'ibai', 'display_name': 'ibai', 'type': '', 'broadcaster_type': 'partner', 'description': 'Si lees esto que sepas que te aprecio', 'profile_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/574228be-01ef-4eab-bc0e-a4f6b68bedba-profile_image-300x300.png', 'offline_image_url': 'https://static-cdn.jtvnw.net/jtv_user_pictures/b01927d9-1cc2-4ba0-b3e2-6e96959179d0-channel_offline_image-1920x1080.jpeg', 'view_count': 0, 'created_at': '2015-02-20T16:47:56Z'}]}
data_dict = data['data'][0]
data_dict
id_canal =  list(data_dict.values())[0]
id_canal

'83232866'

In [6]:
def obtener_vods(id_canal):
    # Calcular la fecha hace un año desde hoy
    fecha_inicio = datetime.now() - timedelta(days=730)
    fecha_inicio_str = fecha_inicio.strftime("%Y-%m-%dT%H:%M:%SZ")

    url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100"

    headers = {
        "Client-ID": client_id,
        "Authorization": f"Bearer {oauth_token}"
    }

    vods_data = []

    while url:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            data = response.json()
            vods = data["data"]

            for vod in vods:
                vod_id = int(vod["id"])
                fecha_inicio = datetime.strptime(vod["created_at"], "%Y-%m-%dT%H:%M:%SZ")
                fecha_final = datetime.strptime(vod["published_at"], "%Y-%m-%dT%H:%M:%SZ")
                duracion = vod["duration"]
                visitas = int(vod["view_count"])
                nombre = vod["title"].lower().replace(' ', '_')
                descripcion = vod["description"]
                url_vod = f"https://www.twitch.tv/videos/{vod_id}"

                vods_data.append({
                    "ID del VOD": vod_id,
                    "URL del VOD": url_vod,
                    "Nombre del VOD": nombre,
                    "Duración": duracion,
                    "Fecha de inicio": fecha_inicio,
                    "Fecha de finalización": fecha_final,
                    "Visitas": visitas,
                    
                })

            pagination = data.get("pagination", None)
            if pagination and "cursor" in pagination:
                cursor = pagination["cursor"]
                url = f"https://api.twitch.tv/helix/videos?user_id={id_canal}&type=archive&started_at={fecha_inicio_str}&first=100&after={cursor}"
            else:
                url = None

        else:
            print("Se produjo un error al realizar la solicitud GET.")
            return None

    # Crear un DataFrame a partir de los datos
    df_vods = pd.DataFrame(vods_data)

    return df_vods

df = obtener_vods(id_canal)
df.info(memory_usage='deep')
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   ID del VOD             67 non-null     int64         
 1   URL del VOD            67 non-null     object        
 2   Nombre del VOD         67 non-null     object        
 3   Duración               67 non-null     object        
 4   Fecha de inicio        67 non-null     datetime64[ns]
 5   Fecha de finalización  67 non-null     datetime64[ns]
 6   Visitas                67 non-null     int64         
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 22.8 KB


,ID del VOD,URL del VOD,Nombre del VOD,Duración,Fecha de inicio,Fecha de finalización,Visitas
0,1891971510,https://www.twitch.tv/videos/1891971510,stream_nocturno_como_en_los_viejos_tiempos_hab...,4h33m50s,2023-08-06 20:56:42,2023-08-06 20:56:42,832214
1,1889973400,https://www.twitch.tv/videos/1889973400,anuncio_el_nuevo_fichaje_de_porcinos_fc_y_me_v...,2h4m28s,2023-08-04 15:47:48,2023-08-04 15:47:48,644479
2,1889092994,https://www.twitch.tv/videos/1889092994,finetwork_koi_vs_movistar_riders_-_semifinales...,4h0m59s,2023-08-03 15:36:08,2023-08-03 15:36:08,1263313
3,1888398387,https://www.twitch.tv/videos/1888398387,si_te_ries_pierdes.,4h11m56s,2023-08-02 19:20:21,2023-08-02 19:20:21,929135
4,1887375965,https://www.twitch.tv/videos/1887375965,ronda_2_-_superliga_-_verano_2023_-_league_of_...,4h53m24s,2023-08-01 15:43:28,2023-08-01 15:43:28,1112979


In [7]:
df.to_csv('twitch_api_kingsleague_csv.csv', index=False)